In [64]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch

from BertExtractors import *

In [65]:
test = pd.read_csv("../data/test.csv")
predictions = pd.read_csv("../results/predictions.csv")
predictions = predictions.rename(columns = {'Unnamed: 0': 'index'})
predictions = predictions.set_index('index')

In [67]:
predictions

,TRUE,vanilla_pred,cont150,top150,cont50,top50,pretrain
index,,,,,,,
25000,0,0,0,0,1,0,0
25001,0,0,0,1,0,0,0
25002,1,1,1,1,1,1,1
25003,1,0,1,1,1,1,1
25004,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
25995,1,1,1,1,1,0,1
25996,1,1,1,1,1,1,1
25997,0,0,1,1,1,1,1


In [71]:
correct150 = pd.DataFrame({'vanilla_pred': (predictions.TRUE == predictions.vanilla_pred),
                          'cont': (predictions.TRUE == predictions.cont150),
                          'top': (predictions.TRUE == predictions.top150)})
correct150.to_csv("../results/correct150.csv")
correct050 = pd.DataFrame({'vanilla_pred': (predictions.TRUE == predictions.vanilla_pred),
                          'cont': (predictions.TRUE == predictions.cont50),
                          'top': (predictions.TRUE == predictions.top50)})
correct050.to_csv("../results/correct050.csv")

In [74]:
pretrain_correct050 = pd.DataFrame({'pretrain': (predictions.TRUE == predictions.pretrain),
                          'cont': (predictions.TRUE == predictions.cont50),
                          'top': (predictions.TRUE == predictions.top50)})
pretrain_correct050.to_csv("../results/pretrain_correct050.csv")

In [38]:
#Analyze the predictions - how many were all right?
def analyze_pred(vanilla, cont, top):
    print("None correct:")
    print(sum((vanilla == 0) & (cont == 0) & (top == 0)))
    print("Only vanilla correct:")
    print(sum((vanilla == 1) & (cont == 0) & (top == 0)))
    print("Only top correct:")
    print(sum((vanilla == 0) & (cont == 0) & (top == 1)))
    print("Only cont correct:")
    print(sum((vanilla == 0) & (cont == 1) & (top == 0)))
    print("Vanilla and cont correct:")
    print(sum((vanilla == 1) & (cont == 1) & (top == 0)))
    print("Vanilla and top correct:")
    print(sum((vanilla == 1) & (cont == 0) & (top == 1)))
    print("Cont and top correct:")
    print(sum((vanilla == 0) & (cont == 1) & (top == 1)))
    print("All correct:")
    print(sum((vanilla == 1) & (cont == 1) & (top == 1)))

Vanilla BERT, k=150

In [39]:
analyze_pred(correct150.vanilla_pred, correct150.cont, correct150.top)

None correct:
67
Only vanilla correct:
44
Only top correct:
41
Only cont correct:
44
Vanilla and cont correct:
97
Vanilla and top correct:
89
Cont and top correct:
68
All correct:
550


Vanilla BERT, k=50

In [46]:
analyze_pred(correct050.vanilla_pred, correct050.cont, correct050.top)

None correct:
48
Only vanilla correct:
84
Only top correct:
51
Only cont correct:
62
Vanilla and cont correct:
146
Vanilla and top correct:
140
Cont and top correct:
59
All correct:
410


Pretrained BERT, k=50

In [75]:
analyze_pred(pretrain_correct050.pretrain, pretrain_correct050.cont, pretrain_correct050.top)

None correct:
12
Only vanilla correct:
120
Only top correct:
12
Only cont correct:
11
Vanilla and cont correct:
197
Vanilla and top correct:
179
Cont and top correct:
14
All correct:
455


## Explanation of All Correct: 25001

In [59]:
extractor = BertTopKExtractor(model_bert, tokenizer, 50)
#The feature representations produced by BERT from the rationales
#Extract from the test
output = []
for i in range(0, 5):
    if i % 50 == 0:
        print(f"Location: {i}")
    ids = extractor.extract(test.iloc[i].review)
    output.append(extractor.input_ids_to_text(ids))

#DFs containing the rationales
rationale_df = pd.DataFrame(output)
#Add the sentiment
rationale_df = pd.concat([rationale_df, test.iloc[:5]['sentiment']],
                          axis = 1, ignore_index = True)
rationale_df.to_csv("../data/text_rationales/top050_test_0005.csv", index = False)
print("Files saved successfully")

Location: 0
Files saved successfully
